In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [136]:
image = cv2.imread('images/receipt_image.jpg', 0)
color = cv2.imread('images/receipt_image.jpg')
hsv = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)

hue, saturation, value = cv2.split(hsv)

In [68]:
def show(image):
    resized_height = 720
    percent = resized_height / len(image)
    resized_width = int(percent * len(image[0]))
    gray = cv2.resize(image,(resized_width,resized_height))


    cv2.imshow('cringe', gray)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [99]:
show(hue)
show(image)

In [185]:
def extract_contours(image, thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    image_copy = image.copy()
    cv2.drawContours(image_copy, contours, -1, (0, 255, 255), 2)

    return image_copy

In [206]:
def extract_largest_rectangular_contour(color_image, preprocessed_image):
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = None
    largest_area = 0
    largest_approx = None
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if area > largest_area:
                largest_area = area
                largest_contour = contour
                largest_approx = approx
                
    if largest_contour is not None:
        cv2.drawContours(color_image, [largest_approx], -1, (0, 255, 0), 3)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        return color_image, largest_approx
    
    return color_image, None

In [211]:
def extract_largest_rectangular_contour_and_perspective(color_image, preprocessed_image):
    contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_contour = None
    largest_area = 0
    largest_approx = None
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:  # Only consider rectangles
            area = cv2.contourArea(contour)
            if area > largest_area:
                largest_area = area
                largest_contour = contour
                largest_approx = approx
                
    if largest_contour is not None and largest_approx is not None:
        cv2.drawContours(color_image, [largest_approx], -1, (0, 255, 0), 3)
        x, y, w, h = cv2.boundingRect(largest_contour)
        cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        
        rect_pts = np.array(largest_approx, dtype="float32")
        
        # Ensure we have exactly 4 points for the rectangle
        if rect_pts.shape[0] == 4:
            rect_pts = order_points(rect_pts)
            width, height = get_perspective_dimensions(rect_pts)
            dst_pts = np.array([
                [0, 0],
                [width - 1, 0],
                [width - 1, height - 1],
                [0, height - 1]
            ], dtype="float32")
            
            M = cv2.getPerspectiveTransform(rect_pts, dst_pts)
            warped_image = cv2.warpPerspective(color_image, M, (width, height))
            return warped_image, largest_approx
        else:
            print("Error: Approximation did not yield 4 points.")
            return color_image, None
    
    return color_image, None

def order_points(pts):
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    ordered_pts = np.zeros((4, 2), dtype="float32")
    
    ordered_pts[0] = pts[np.argmin(s)]  # Top-left point
    ordered_pts[2] = pts[np.argmax(s)]  # Bottom-right point
    ordered_pts[1] = pts[np.argmin(diff)]  # Top-right point
    ordered_pts[3] = pts[np.argmax(diff)]  # Bottom-left point
    
    return ordered_pts

def get_perspective_dimensions(pts):
    width1 = np.linalg.norm(pts[0] - pts[1])
    width2 = np.linalg.norm(pts[2] - pts[3])
    max_width = max(int(width1), int(width2))
    
    height1 = np.linalg.norm(pts[0] - pts[3])
    height2 = np.linalg.norm(pts[1] - pts[2])
    max_height = max(int(height1), int(height2))
    
    return max_width, max_height

In [213]:
im, cor = extract_largest_rectangular_contour_and_perspective(color, dilated)
show(im)

IndexError: index 5 is out of bounds for axis 0 with size 4

In [208]:
im, cor = extract_largest_rectangular_contour(color, dilated)
show(im)
print(cor)

[[[ 330  386]]

 [[ 306 1218]]

 [[ 701 1221]]

 [[ 708  370]]]


In [186]:
show(extract_contours(color, dilated))

In [184]:

thresh = cv2.threshold(hue, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
show(thresh)

eroded = cv2.erode(thresh, kernel=None, iterations=7)
show(eroded)
dilated = cv2.dilate(eroded, kernel=None, iterations=15)
show(dilated)

cv2.imwrite('temp.jpg', dilated)



True

In [181]:
def extract_rectangle_from_edges(color_image, binary_image):
    edges = binary_image
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
    color_image = color_image.copy()
    
    horizontal_lines = []
    vertical_lines = []
    
    if lines is not None:
        lines = np.squeeze(lines)

        for line in lines:
            x1, y1, x2, y2 = line
            # Calculate the angle of the line using the arctangent of the slope
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi  # Convert angle to degrees
            
            # Classify the line based on its angle
            if 0 <= angle <= 20 or 340 <= angle <= 360:
                vertical_lines.append(line)
            elif 160 <= angle <= 200:  # Horizontal lines (angle close to 0 or 180 degrees)
                horizontal_lines.append(line)
        print(horizontal_lines)
        print(vertical_lines)
        if len(horizontal_lines) >= 2 and len(vertical_lines) >= 2:
            x1, y1, x2, y2 = horizontal_lines[0]
            x3, y3, x4, y4 = horizontal_lines[1]
            x5, y5, x6, y6 = vertical_lines[0]
            x7, y7, x8, y8 = vertical_lines[1]
            
            rectangle_points = [(x1, y1), (x3, y3), (x5, y5), (x7, y7)]
            
            rectangle_points = np.array(rectangle_points, dtype=np.int32)
            cv2.polylines(color_image, [rectangle_points], isClosed=True, color=(0, 255, 0), thickness=2)
            
            for line in horizontal_lines + vertical_lines:
                x1, y1, x2, y2 = line
                cv2.line(color_image, (x1, y1), (x2, y2), (255, 0, 0), 2)
    
    return color_image

In [182]:
show(extract_rectangle_from_edges(color, dilated))

[]
[array([  24, 1195,  124, 1231], dtype=int32), array([  24, 1197,  124, 1233], dtype=int32), array([ 361, 1321,  551, 1382], dtype=int32), array([ 434, 1348,  555, 1390], dtype=int32), array([779, 238, 976, 291], dtype=int32), array([779, 215, 965, 268], dtype=int32), array([475, 135, 583, 169], dtype=int32), array([ 401, 1338,  559, 1393], dtype=int32), array([ 722, 1434,  843, 1473], dtype=int32), array([780, 205, 948, 260], dtype=int32), array([ 361, 1322,  555, 1389], dtype=int32), array([779, 233, 969, 280], dtype=int32), array([ 372, 1299,  534, 1348], dtype=int32), array([ 730, 1429,  846, 1464], dtype=int32), array([474, 127, 581, 158], dtype=int32), array([474, 123, 579, 155], dtype=int32), array([ 361, 1315,  472, 1351], dtype=int32), array([779, 207, 947, 261], dtype=int32), array([474, 118, 575, 149], dtype=int32), array([779, 210, 965, 270], dtype=int32), array([ 377, 1296,  534, 1344], dtype=int32), array([779, 219, 965, 273], dtype=int32), array([ 675, 1466,  854, 153

In [180]:
show(color)

In [114]:
im = extract_contours(color, dilated)
show(im)

In [125]:
show(dilated)

In [205]:
eroded = cv2.erode(dilated, kernel=None, iterations=100)
show(eroded)

dilated = cv2.dilate(eroded, kernel=None, iterations=100)
show(extract_contours(color, dilated))

In [ ]:
def filter_contours_and_leave_only_rectangles(self):
    self.rectangular_contours = []
    for contour in self.contours:
        peri = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if len(approx) == 4:
            self.rectangular_contours.append(approx)
    # Below lines are added to show all rectangular contours
    # This is not needed, but it is useful for debugging
    self.image_with_only_rectangular_contours = self.image.copy()
    cv2.drawContours(self.image_with_only_rectangular_contours, self.rectangular_contours, -1, (0, 255, 0), 3)

In [73]:
adaptive_thresh = cv2.adaptiveThreshold(hue, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
show(adaptive_thresh)

In [72]:
# show(adaptive_thresh)

median_blur = cv2.medianBlur(adaptive_thresh, ksize=7)
show(median_blur)

gaussian = cv2.GaussianBlur(median_blur, (15,15), 0)
show(gaussian)

In [44]:
eroded = cv2.erode(gaussian, kernel=None, iterations=1)
dilated = cv2.dilate(eroded, kernel=None, iterations=2)
show(dilated)

eroded = cv2.erode(dilated, kernel=None, iterations=1)
dilated = cv2.dilate(eroded, kernel=None, iterations=2)


eroded = cv2.erode(dilated, kernel=None, iterations=1)
dilated = cv2.dilate(eroded, kernel=None, iterations=2)


eroded = cv2.erode(dilated, kernel=None, iterations=1)
dilated = cv2.dilate(eroded, kernel=None, iterations=2)


eroded = cv2.erode(dilated, kernel=None, iterations=1)
dilated = cv2.dilate(eroded, kernel=None, iterations=2)


eroded = cv2.erode(dilated, kernel=None, iterations=1)
dilated = cv2.dilate(eroded, kernel=None, iterations=2)

eroded = cv2.erode(dilated, kernel=None, iterations=1)
dilated = cv2.dilate(eroded, kernel=None, iterations=2)
show(dilated)

In [52]:
eroded = cv2.erode(dilated, kernel=None, iterations=12)
show(eroded)

In [50]:
sift = cv2.SIFT_create()
kp = sift.detect(eroded,None)


img2 = cv2.drawKeypoints(image.copy(), kp, None, color=(255,0,0))

show(img2)